# Projet : Traitement Automatique des Langues (Partie 1)

Nous avons à notre disposition deux fichiers CSV ([allocine_genres_test.csv](data/allocine_genres_test.csv) et [allocine_genres_train.csv](data/allocine_genres_train.csv)) contenant des informations sur des films et leurs genres. Le but de ce projet est de prédire les genres d'un film à partir de son synopsis notamment (et d'autres informations).

L’objectif est d’entraîner un outil de classification automatique des films en fonction de leur genre. La classification doit se baser sur le texte de la synopsis et sur le titre des films. Le texte et le titre des articles ont déjà été tokenisés et tous les tokens sont séparés par un espace.

## 1. Importation des données et analyse exploratoire

Les données sont disponibles dans le dossier [data](data/). Nous allons commencer par importer les données et les analyser.

In [1]:
%%capture

# exécuter cette cellule pour installer les dépendances et télécharger les modèles spacy
# remplacer `python` par `python3` si nécessaire
!python -m pip install --upgrade -r requirements.txt
!python -m spacy download fr_core_news_sm

In [1]:
import os
import sys

import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
import spacy

from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from spacy import displacy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.linear_model import LogisticRegression

train: pd.DataFrame = pd.read_csv(os.path.join('data', 'allocine_genres_train.csv'))

In [2]:
print(train.shape)
print(train.head())

(2875, 22)
   Unnamed: 0         acteur_1           acteur_2           acteur_3   
0        4772    Albert Finney      Lauren Bacall  Jacqueline Bisset  \
1         335      Henry Fonda      Martin Balsam       John Fiedler   
2        4860   Alexandra Lamy  Michaël Abiteboul       Julia Piaton   
3        1913  Charles Chaplin  Virginia Cherrill        Harry Myers   
4        3726   Robert De Niro   Cuba Gooding Jr.    Charlize Theron   

   allocine_id  annee_prod  annee_sortie  box_office_fr  couleur  duree  ...   
0         1453        1974        1975.0       549055.0  Couleur  128.0  ...  \
1         4063        1957        1957.0            NaN      NaN   95.0  ...   
2       241952        2016        2019.0            NaN  Couleur   90.0  ...   
3         2256        1931        1931.0            NaN      NaN   87.0  ...   
4        27434        2000        2001.0       221677.0  Couleur  129.0  ...   

  nb_critiques_presse nb_critiques_spectateurs  nb_notes_spectateurs   
0  

In [3]:
print(train['genre'].value_counts(), end='\n\n')
print(random.choice(train['synopsis'].unique()))

genre
drame              501
comédie            483
romance            443
policier           331
horreur            299
science fiction    298
biopic             191
documentaire       167
historique         162
Name: count, dtype: int64

Sète , 1994 . Amin , apprenti scénariste installé à Paris , retourne un été dans sa ville natale , pour retrouver famille et amis d’ enfance . Accompagné de son cousin Tony et de sa meilleure amie Ophélie , Amin passe son temps entre le restaurant de spécialités tunisiennes tenu par ses parents , les bars de quartier , et la plage fréquentée par les filles en vacances . Fasciné par les nombreuses figures féminines qui l’ entourent , Amin reste en retrait et contemple ces sirènes de l’ été , contrairement à son cousin qui se jette dans l’ ivresse des corps . Mais quand vient le temps d’ aimer , seul le destin - le mektoub - peut décider .


On remarque déjà que les donnés chiffrées sont soit des entiers soit des flottants. Les données textuelles sont des chaînes de caractères. Certaines données sont manquantes : `NaN` dans le cas des données chiffrées et une chaîne vide dans le cas des données textuelles.

Pour traiter les données manquantes, nous avons deux solutions :

1. Dans le cas où il y a très peu de données manquantes, on peut simplement supprimer les entrées qui contiennent ces données manquantes. Cela peut être acceptable si le nombre de données manquantes est très faible par rapport à la taille de l'ensemble de données et que la suppression de ces entrées n'affecte pas significativement les résultats de l'analyse.
2. En revanche, si le nombre de données manquantes est important, la suppression de ces entrées pourrait entraîner une perte d'informations importantes pour l'analyse. Dans ce cas, il est généralement préférable de remplacer les valeurs manquantes par une valeur qui représente au mieux l'information manquante. Par exemple, si les données manquantes sont des scores au box-office, vous pouvez remplacer ces données manquantes par la moyenne ou la médiane des scores de box-office disponibles dans les données.


In [4]:
EMPTY_TOKEN = '<EMPTY>'

# remove rows with missing values
train_dataset_1 = train.dropna(how='any', inplace=False)

# replace missing values with either the mean or the median (or empty token)
list_headers = train.columns.values.tolist()
list_of_numerical_headers = train._get_numeric_data().columns.values.tolist()
list_of_categorical_headers = list(set(list_headers) - set(list_of_numerical_headers))

train_dataset_2 = train.copy()
for header in list_of_numerical_headers:
  train_dataset_2[header].fillna(train_dataset_2[header].median(), inplace=True)
for header in list_of_categorical_headers:
  train_dataset_2[header].fillna(EMPTY_TOKEN, inplace=True)

print(train_dataset_1.shape)
print(train_dataset_2.shape)

(891, 22)
(2875, 22)


Lorsqu'on supprime simplement toutes les entrées où il manque au moins une valeur, on se retrouve uniquement avec 891 valeurs en tout. Cela signifie que nous avons perdu beaucoup d'informations. Nous allons donc utiliser la deuxième solution (au moins dans un premier temps) et remplacer les valeurs manquantes par des valeurs qui représentent au mieux l'information manquante.

## 2. Prétraitement des données

Il faut aussi corriger les entrées textuelles, ainsi qu'appliquer un certain nombre d'algorithmes de prétraitement comme : la suppression des caractères spéciaux, la suppression des stop words, la suppression des mots trop fréquents ou trop rares, la lemmatisation, la suppression des mots trop longs, etc.

On définit donc un ensemble de fonctions et de filtres qui vont nous permettre de prétraiter les données textuelles.

In [5]:
nltk.download('stopwords', quiet=True)        # download the stopwords corpus
nlp = spacy.load('fr_core_news_sm')           # load the French model
fr_stopwords = set(stopwords.words('french')) # so that `in` tests are faster
stemmer = FrenchStemmer()                     # for stemming words


# get the tokens of a sentence (word based tokenization)
def get_tokens_words(text: str) -> list[str]:
  doc = nlp(text)
  return [w.text for w in doc]


# remove stopwords from a sentence
def clean_sentence(text: str) -> list[str]:
  clean_words: list[str] = []
  for token in get_tokens_words(text):
    if token not in fr_stopwords:
      clean_words.append(token)
  return clean_words


# get the tokens of multiple sentences (sentence based tokenization)
def get_tokens_sentences(text: str) -> list[str]:
  doc = nlp(text)
  return [s.text for s in doc.sents]


# get the lemmas of a sentence
def get_stem(text: str) -> list[str]:
  doc = nlp(text)
  return [stemmer.stem(w.text) for w in doc]


# get the named entities of a sentence
def get_ner(text: str) -> list[str]:
  doc = nlp(text)
  return [(ent.text, ent.label_) for ent in doc.ents]


# render the named entities of a sentence in a Jupyter notebook
def render_ner(text: str) -> None:
  doc = nlp(text)
  displacy.render(doc, style='ent', jupyter=True)


# get the part of speech of a sentence
def get_pos(text: str) -> list[str]:
  doc = nlp(text)
  return [(token, token.pos_) for token in doc]


# render the part of speech of a sentence in a Jupyter notebook
def render_pos(text: str) -> None:
  doc = nlp(text)
  displacy.render(doc, style='dep', options={'distance': 90})


# get the word embeddings of a sentence
def get_word_embeddings(text: str) -> list[np.ndarray]:
  doc = nlp(text)
  return [token.vector for token in doc]


# get the similarity between two sentences
def get_mean_embedding(text1: str, text2: str) -> float:
  doc1 = nlp(text1)
  doc2 = nlp(text2)
  mean1 = np.mean([token.vector for token in doc1], axis=0)
  mean2 = np.mean([token.vector for token in doc2], axis=0)

  return np.dot(mean1, mean2) / (np.linalg.norm(mean1) * np.linalg.norm(mean2))

Juste pour clarifier les choses, nous allons simplement effectuer des tests sur l'ensemble de phrases suivantes :

1. "Le réseau sera bientôt rétabli à Marseille"
2. "La panne réseau affecte plusieurs utilisateurs de l'opérateur"
3. "Il fait 18 degrés ici"
4. "Bouygues a eu une coupure de réseau à Marseille. La panne a affecté 300.000 utilisateurs."

In [6]:
text1 = 'Le réseau sera bientôt rétabli à Marseille.'
text2 = 'La panne réseau affecte plusieurs utilisateurs de l\'opérateur'
text3 = 'Il fait 18 degrés ici'
text4 = 'Bouygues a eu une coupure de réseau à Marseille. La panne a affecté 300.000 utilisateurs.'

In [7]:
# basic tokenization
# we can observe `get_tokens_sentences` do not "cut" at each . or ! or ?

print(get_tokens_words(text1))
print(clean_sentence(text1))
print(get_tokens_sentences(text4))

['Le', 'réseau', 'sera', 'bientôt', 'rétabli', 'à', 'Marseille', '.']
['Le', 'réseau', 'bientôt', 'rétabli', 'Marseille', '.']
['Bouygues a eu une coupure de réseau à Marseille.', 'La panne a affecté 300.000 utilisateurs.']


In [8]:
# stemming
# this doesn't work very well for French...

print(get_stem(text1))
print(get_stem(text2))

['le', 'réseau', 'ser', 'bientôt', 'rétabl', 'à', 'marseil', '.']
['la', 'pann', 'réseau', 'affect', 'plusieur', 'utilis', 'de', "l'", 'oper']


In [9]:
# named entities recognition

print(get_ner(text4))
render_ner(text4)

[('Bouygues', 'ORG'), ('Marseille', 'LOC')]


In [10]:
# part of speech

print(get_pos(text1))
render_pos(text1)

[(Le, 'DET'), (réseau, 'NOUN'), (sera, 'AUX'), (bientôt, 'ADV'), (rétabli, 'ADJ'), (à, 'ADP'), (Marseille, 'PROPN'), (., 'PUNCT')]


In [11]:
# word embeddings and mean embedding (similarity)

print(get_word_embeddings(text1)[0].shape)
print(get_mean_embedding(text1, text2))
print(get_mean_embedding(text1, text4))
print(get_mean_embedding(text2, text4))

(96,)
0.29973912
0.39468837
0.53404236


## 3. Préparation des données

Nous allons maintenant préparer les données pour l'entraînement de notre modèle.

Nous allons donc appliquer les fonctions de prétraitement sur les données textuelles et transformer les données chiffrées en données numériques. Dans un premier temps, nous confectionnerons des ensembles contenant toutes les valeurs et les informations présentes dans les données. Nous verrons par la suites lesquelles sont les plus pertinantes en fonction des résultats obtenus et de nos modèles.

In [25]:
# we will use train_dataset_2 since it has no missing values

values = ['synopsis', 'nb_critiques_presse', 'nb_critiques_spectateurs', 'realisateurs', 'titre']
X = train_dataset_2[values]
y = train_dataset_2['genre']

print(X.shape)
print(X.head())
print(y.shape)
print(y.head())

(2875, 5)
                                            synopsis  nb_critiques_presse   
0  En visite à Istanbul , le célèbre détective be...                 19.0  \
1  Un jeune homme d' origine modeste est accusé d...                  7.0   
2  Lorsque Marie-Laure , mère de quatre jeunes en...                 19.0   
3  Un vagabond s’ éprend d’ une belle et jeune ve...                  4.0   
4  L' histoire vraie de Carl Brashear , premier A...                 17.0   

   nb_critiques_spectateurs        realisateurs   
0                     125.0        Sidney Lumet  \
1                     771.0        Sidney Lumet   
2                      22.0       Nicolas Cuche   
3                     190.0     Charles Chaplin   
4                     104.0  George Tillman Jr.   

                             titre  
0  Le Crime de l' Orient - Express  
1              12 hommes en colère  
2             Après moi le bonheur  
3         Les Lumières de la ville  
4        Les Chemins de la dignité 

In [26]:
# separate the dataset into a training set and a validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12, shuffle=True)

Pour la suite, nous définirons des pipelines de traitement spécifiques à chaque type de colonne. En particulier, les colonnes correspondant à des textes (`list_of_categorical_headers`) dans `X` seront vectorisées.

Pour toutes les données dans `list_of_categorical_headers`, les tokens ont déjà été séparés par des espaces. Nous allons utiliser `TfidfVectorizer` pour vectoriser ces données.

In [27]:
tfidf_vectorizer = TfidfVectorizer(
  analyzer='word',
  tokenizer=lambda x: str.split(x, sep=' '),
  token_pattern=None,
  lowercase=True,
  stop_words=list(fr_stopwords),
  min_df=0.01,
  max_df=0.95,
)


In [28]:
res = tfidf_vectorizer.fit_transform(X_train['synopsis'])
bow = pd.DataFrame(res.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
print(bow.shape)
print(bow.head())

(2300, 684)
     !    "         (         )         -       ...   18   20        30   
0  0.0  0.0  0.000000  0.000000  0.178501  0.073004  0.0  0.0  0.000000  \
1  0.0  0.0  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.000000   
2  0.0  0.0  0.190019  0.190019  0.000000  0.000000  0.0  0.0  0.000000   
3  0.0  0.0  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.208605   
4  0.0  0.0  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.000000   

          :  ...  étrange  étranges  études  étudiant  événements  être   
0  0.078143  ...      0.0       0.0     0.0       0.0    0.161489   0.0  \
1  0.188918  ...      0.0       0.0     0.0       0.0    0.000000   0.0   
2  0.103920  ...      0.0       0.0     0.0       0.0    0.000000   0.0   
3  0.196572  ...      0.0       0.0     0.0       0.0    0.000000   0.0   
4  0.150172  ...      0.0       0.0     0.0       0.0    0.000000   0.0   

   êtres  île    –         …  
0    0.0  0.0  0.0  0.000000  
1    0.0  0.0  0.0  0.00

Nous pouvons aussi utiliser ce `vectorizer` pour extraire des statistiques sur les données textuelles. Par exemple, la longueur en nombre de caractères, le nombre de phrases, ...

In [29]:
def make_stats(texts: list[str]) -> list[dict[str, int]]:
  return [{
    'len': len(t),
    'nb_sentences': t.count('.') + t.count('!') + t.count('?'),
  } for t in texts]

stats_transformer = FunctionTransformer(make_stats, validate=False)
stats_vectorizer = DictVectorizer(sparse=False)

In [30]:
res = stats_vectorizer.fit_transform(stats_transformer.transform(X_train['synopsis']))
stats = pd.DataFrame(res, columns=stats_vectorizer.get_feature_names_out())

print(stats.shape)
print(stats.head())

(2300, 2)
     len  nb_sentences
0  922.0           8.0
1  759.0           9.0
2  679.0           3.0
3  626.0           8.0
4  281.0           2.0


On normalise les données en utilisant `MinMaxScaler` pour les données dans notre dictionnaire

In [31]:
min_max_scaler = MinMaxScaler()
scaled_stats = pd.DataFrame(min_max_scaler.fit_transform(stats), columns=stats.columns)

print(scaled_stats.shape)
print(scaled_stats.head())

(2300, 2)
        len  nb_sentences
0  0.463821      0.363636
1  0.378969      0.409091
2  0.337324      0.136364
3  0.309735      0.363636
4  0.130141      0.090909


On rajoute aussi de quoi faire un encodage one-hot des données catégorielles.

In [41]:
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [42]:
res = one_hot_encoder.fit_transform(X_train[['realisateurs']])
realisateurs = pd.DataFrame(res, columns=one_hot_encoder.get_feature_names_out())

print(realisateurs.shape)
print(realisateurs.head())

(2300, 1531)
   realisateurs_Aaron Seltzer, Jason Friedberg  realisateurs_Abbas Kiarostami   
0                                          0.0                            0.0  \
1                                          0.0                            0.0   
2                                          0.0                            0.0   
3                                          0.0                            0.0   
4                                          0.0                            0.0   

   realisateurs_Abd Al Malik  realisateurs_Abdelhamid Bouchnak   
0                        0.0                               0.0  \
1                        0.0                               0.0   
2                        0.0                               0.0   
3                        0.0                               0.0   
4                        0.0                               0.0   

   realisateurs_Abdellatif Kechiche  realisateurs_Abderrahmane Sissako   
0                            

## 4. Création de la pipeline

Nous allons maintenant procéder à la création de la pipeline en combinant les chaînes de pré-traitement.

In [64]:
column_transformer = ColumnTransformer(
  [
    # 'synopsis' column : tf-idf vectorization
    ('synopsis', tfidf_vectorizer, 'synopsis'),

    # 'nb_critiques_presse' and 'nb_critiques_spectateurs' columns : min-max scaling
    ('min_max', min_max_scaler, ['nb_critiques_presse', 'nb_critiques_spectateurs']),
    # 'realisateurs' column : one-hot encoding
    ('realisateurs', one_hot_encoder, ['realisateurs']),
    # 'titre' column : tf-idf vectorization
    # ('titre', tfidf_vectorizer, 'titre'),
  ],
  remainder='drop', # drop the columns not specified
)

In [65]:
# learning
classifier_pipeline = make_pipeline(column_transformer, LogisticRegression(max_iter=1000))

In [66]:
# fit
classifier_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('synopsis',
                                                  TfidfVectorizer(max_df=0.95,
                                                                  min_df=0.01,
                                                                  stop_words=['aient',
                                                                              'étés',
                                                                              'au',
                                                                              'aviez',
                                                                              'on',
                                                                              'te',
                                                                              'ayantes',
                                                                              'eurent',
                                                                              'mais',
                                                                              'eux',
                                                                              'toi',
                                                                              'étées',
                                                                              'soyons',
                                                                              'aura',
                                                                              'fût',
                                                                              'seraient',
                                                                              'elle',
                                                                              'nos',
                                                                              'pour',
                                                                              'eusse',
                                                                              'en',
                                                                              'c',
                                                                              'd',
                                                                              'étant',
                                                                              'sois',
                                                                              'son',
                                                                              'mes',
                                                                              'eut',
                                                                              'sommes',
                                                                              'dans', ...],
                                                                  token_pattern=None,
                                                                  tokenizer=<function <lambda> at 0x7f64dc4a1580>),
                                                  'synopsis'),
                                                 ('min_max', MinMaxScaler(),
                                                  ['nb_critiques_presse',
                                                   'nb_critiques_spectateurs']),
                                                 ('realisateurs',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['realisateurs'])])),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [67]:
# predict
y_pred = classifier_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

         biopic       0.50      0.12      0.19        43
        comédie       0.52      0.72      0.60       100
   documentaire       0.85      0.28      0.42        40
          drame       0.43      0.55      0.48       112
     historique       0.71      0.37      0.49        27
        horreur       0.59      0.49      0.54        59
       policier       0.55      0.58      0.56        57
        romance       0.41      0.50      0.45        78
science fiction       0.68      0.58      0.62        59

       accuracy                           0.51       575
      macro avg       0.58      0.46      0.48       575
   weighted avg       0.54      0.51      0.50       575

